<a href="https://colab.research.google.com/github/andreweuw/ASCI_ART/blob/master/docs/biom3d_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@markdown ## First check that your Runtime is in GPU mode (you can run this cell to do so)

#@markdown If not, go to `Runtime` > `Change runtime type` > `Hardware accelerator` > `GPU`

!nvidia-smi

Wed Aug  6 18:53:23 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   48C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
#@markdown ##Install Biom3d
# !pip3 uninstall torch -y # bug fix, compatibility problem between torchIO and pytorch>=2.3
# !pip3 install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113
!pip3 install biom3d==0.0.46 torchio==0.20.5 deprecated --no-deps
!pip3 install SimpleITK paramiko netcat appdirs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.9/207.9 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.8/175.8 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.9/223.9 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.3/160.3 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 67.5 MB/s eta 0:00:00


In [3]:
#@markdown ##Import the necessary Python library
import os
import biom3d
from biom3d.train import train
from biom3d.pred import pred
from biom3d.eval import eval

In [4]:
#@markdown ##Mount your Google drive

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#@markdown ## Define input and output folders

#@markdown **Where training data will be saved?** This will include two folders: the config folder (containing configuration files) and the logs folder (containing the model folders)

biom3d_dir = '/content/gdrive/MyDrive/biom3d'#@param {type:"string"}
model_name = "unet_default" #@param {type:"string"}

#@markdown **Where are your images and masks?**
img_dir = "" #@param {type:"string"}
msk_dir = "" #@param {type:"string"}

#@markdown **How many classes are there in your annotations?**

num_classes = 1 #@param {type:"number"}

#@markdown **How long would you like to train your model?**
num_epochs = 2 #@param {type:"number"}

In [ ]:
#@markdown ## Preprocess your data before training
config_path = biom3d.preprocess.auto_config_preprocess(
    img_dir=img_dir,
    msk_dir=msk_dir,
    num_classes=num_classes,
    desc=model_name,
    config_dir=os.path.join(biom3d_dir, "configs/"),
    base_config=None,
    ct_norm=False,
    num_epochs=num_epochs,
    num_workers=2,
    logs_dir=os.path.join(biom3d_dir, "logs/"),
)

In [ ]:
#@markdown ## Start the training

#@markdown If you want to use of different configuration file or use an existing one, complete the following field. **Leave this field empty if you want to use the config file obtained during the preprocessing.**

custom_config_path = "" #@param {type:"string"}

if len(custom_config_path) > 0:
  config_path = custom_config_path

print("We will the following configuration file for training:", config_path)
builder = train(config=config_path)

In [ ]:
#@markdown ## Make predictions with your trained model

#@markdown **Path to the model folder: leave it empty if you want to use the model obtained during the previous training step.** This should look like `/content/gdrive/MyDrive/biom3d/logs/20230602-162331-unet_default_fold0`.

custom_log_path = "/content/gdrive/MyDrive/biom3d_Franzi/logs/3less-sensitive20240321-020005-unet_default_fold0" #@param {type:"string"}

if len(custom_log_path) > 0:
  log=custom_log_path
else:
  assert 'builder' in locals().keys(), "No existing model folder found. Please complete the `custom_log_path` field or train a model."
  log = builder.base_dir

#@markdown **Prediction input directory:**
pred_dir_in = "/content/gdrive/MyDrive/raw/Freddy/Spongy Mesophyll-25-04-14.lif_flattened" #@param {type:"string"}

#@markdown **Prediction output directory:** (where the prediction masks will be stored)
pred_dir_out = "/content/gdrive/MyDrive/predictions/FreddySponges"#@param {type:"string"}

dir_out = pred(
    log=log,
    dir_in=pred_dir_in,
    dir_out=pred_dir_out
    )


UNet(
  (encoder): VGGEncoder(
    (layers): ModuleList(
      (0): Sequential(
        (0): EncoderBlock(
          (conv1): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
          (bn1): InstanceNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
          (conv2): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
          (bn2): InstanceNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
        )
      )
      (1): Sequential(
        (0): EncoderBlock(
          (conv1): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 2, 2), padding=(1, 1, 1), bias=False)
          (bn1): InstanceNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
          (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
          (bn2): InstanceNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Input shape: (1, 13, 512, 512)
Preprocessed shape: (1, 13, 512, 512)
AMP enabled


100%|██████████| 16/16 [00:26<00:00,  1.67s/it]


Prediction done!
Aggregation...
Aggregation done!
Model output shape: torch.Size([2, 13, 512, 512])
Post-processing...
Post-processing done!
Output shape: (13, 512, 512)
Saving images in /content/gdrive/MyDrive/predictions/FreddySponges/3less-sensitive20240321-020005-unet_default_fold0/_Spongy Mesophyll-25-04-14 - Spongy Mesophyll MOCK- 001_FRAP Pb1 Series44_Channel_1_Time_1.tif
running prediction for image:  /content/gdrive/MyDrive/raw/Freddy/Spongy Mesophyll-25-04-14.lif_flattened/_Spongy Mesophyll-25-04-14 - Spongy Mesophyll MOCK- 001_FRAP Pb1 Series44_Channel_1_Time_10.tif
Input shape: (1, 13, 512, 512)
Preprocessed shape: (1, 13, 512, 512)
AMP enabled


...

100%|██████████| 16/16 [00:28<00:00,  1.78s/it]


Prediction done!
Aggregation...
Aggregation done!
Model output shape: torch.Size([2, 13, 512, 512])
Post-processing...
Post-processing done!
Output shape: (13, 512, 512)
Saving images in /content/gdrive/MyDrive/predictions/FreddySponges/3less-sensitive20240321-020005-unet_default_fold0/_Spongy Mesophyll-25-04-14 - Spongy Mesophyll MOCK- 001_FRAP Pb1 Series44_Channel_1_Time_2.tif
running prediction for image:  /content/gdrive/MyDrive/raw/Freddy/Spongy Mesophyll-25-04-14.lif_flattened/_Spongy Mesophyll-25-04-14 - Spongy Mesophyll MOCK- 001_FRAP Pb1 Series44_Channel_1_Time_20.tif
Input shape: (1, 13, 512, 512)
Preprocessed shape: (1, 13, 512, 512)
AMP enabled


100%|██████████| 16/16 [00:28<00:00,  1.78s/it]


Prediction done!
Aggregation...
Aggregation done!
Model output shape: torch.Size([2, 13, 512, 512])
Post-processing...
Post-processing done!
Output shape: (13, 512, 512)
Saving images in /content/gdrive/MyDrive/predictions/FreddySponges/3less-sensitive20240321-020005-unet_default_fold0/_Spongy Mesophyll-25-04-14 - Spongy Mesophyll MOCK- 001_FRAP Pb1 Series44_Channel_1_Time_20.tif
running prediction for image:  /content/gdrive/MyDrive/raw/Freddy/Spongy Mesophyll-25-04-14.lif_flattened/_Spongy Mesophyll-25-04-14 - Spongy Mesophyll MOCK- 001_FRAP Pb1 Series44_Channel_1_Time_21.tif
Input shape: (1, 13, 512, 512)
Preprocessed shape: (1, 13, 512, 512)
AMP enabled


 25%|██▌       | 4/16 [00:07<00:22,  1.84s/it]

In [ ]:
#@markdown ## Evaluate your model on a test set

#@markdown **Prediction output directory:** where the previous prediction masks have been stored and the number of classes in your images. Leave the default values if you want to use the previous prediction path.
new_pred_dir_out = ""#@param {type:"string"}
new_num_classes = 0 #@param {type:"number"}

if new_num_classes == 0:
  assert 'num_classes' in locals().keys(), "Number of classes equal to zero and the previous number of classes does not exist. Please provide one."
  new_num_classes = num_classes

if len(new_pred_dir_out) == 0:
  assert len(dir_out) > 0, "Prediction path seems to be empty and no previous path detected."
  new_pred_dir_out = dir_out

#@markdown **Path to test masks** The test masks must correspond to the predictions.
test_msk_dir = "" #@param {type:"string"}

eval(
    dir_lab=test_msk_dir,
    dir_out=new_pred_dir_out,
    num_classes=new_num_classes
)